<h1>SQL Analysis on Strava Fitness App</h1>

<h3>Problem Statement</h3>

The goal is to analyze detailed user data on activity, sleep, heart rate, and weight to uncover insights into health and fitness patterns. This involves integrating multiple data sources to explore relationships between physical activity, sleep quality, calories burned, and body metrics. By applying advanced SQL techniques such as joins, aggregations, and window functions, the analysis aims to identify trends, correlations, and rankings that support data-driven fitness and wellness recommendations.



<h3>Step 1: Load All CSVs into SQLite </h3>

In [2]:
import pandas as pd
import sqlite3
import os

# Connect or create SQLite DB
conn = sqlite3.connect('fitbit.db')

# CSV file paths
base_path = '/home/prasanna/Downloads/strava_csvfiles'
csv_files = {
    "dailyActivity": "dailyActivity_merged.csv",
    "dailyCalories": "dailyCalories_merged.csv",
    "dailyIntensities": "dailyIntensities_merged.csv",
    "dailySteps": "dailySteps_merged.csv",
    "heartRate": "heartrate_seconds_merged.csv",
    "hourlyCalories": "hourlyCalories_merged.csv",
    "hourlyIntensities": "hourlyIntensities_merged.csv",
    "hourlySteps": "hourlySteps_merged.csv",
    "minuteCaloriesNarrow": "minuteCaloriesNarrow_merged.csv",
    "minuteCaloriesWide": "minuteCaloriesWide_merged.csv",
    "minuteIntensitiesNarrow": "minuteIntensitiesNarrow_merged.csv",
    "minuteIntensitiesWide": "minuteIntensitiesWide_merged.csv",
    "minuteMETsNarrow": "minuteMETsNarrow_merged.csv",
    "minuteSleep": "minuteSleep_merged.csv",
    "minuteStepsNarrow": "minuteStepsNarrow_merged.csv",
    "minuteStepsWide": "minuteStepsWide_merged.csv",
    "sleepDay": "sleepDay_merged.csv",
    "weightLog": "weightLogInfo_merged.csv"
}

# Load each CSV into a SQL table
for table, file in csv_files.items():
    df = pd.read_csv(os.path.join(base_path, file))
    df.to_sql(table, conn, if_exists='replace', index=False)
    print(f"Loaded: {table}")

conn.commit()


Loaded: dailyActivity
Loaded: dailyCalories
Loaded: dailyIntensities
Loaded: dailySteps
Loaded: heartRate
Loaded: hourlyCalories
Loaded: hourlyIntensities
Loaded: hourlySteps
Loaded: minuteCaloriesNarrow
Loaded: minuteCaloriesWide
Loaded: minuteIntensitiesNarrow
Loaded: minuteIntensitiesWide
Loaded: minuteMETsNarrow
Loaded: minuteSleep
Loaded: minuteStepsNarrow
Loaded: minuteStepsWide
Loaded: sleepDay
Loaded: weightLog


<h3>Step2:Analysis Using all SQL functions</h3>

<h3>1. Sleep variation per user</h3>

In [11]:
query = """
SELECT Id, AVG(TotalMinutesAsleep) AS avg_sleep, 
       MAX(TotalMinutesAsleep) - MIN(TotalMinutesAsleep) AS sleep_variation
FROM sleepDay
GROUP BY Id
ORDER BY sleep_variation DESC
"""
pd.read_sql_query(query, conn)


,Id,avg_sleep,sleep_variation
0,1644430081,294.000000,677
1,4319703577,476.653846,633
2,1927972279,417.000000,584
3,4388161847,403.125000,557
4,5577150313,432.000000,529
5,1503960366,360.280000,455
6,5553957443,463.483871,453
7,4020332650,349.375000,424
8,8053475328,297.000000,412
9,4445114986,385.178571,404


Explanation:

This query calculates the average sleep duration (avg_sleep) and the sleep variation (sleep_variation) for each user (Id) from the sleepDay dataset. The sleep variation is computed as the difference between the maximum and minimum minutes slept by a user, reflecting how consistent their sleep patterns are. The results are ordered by sleep variation in descending order, highlighting users with the most irregular sleep durations at the top.

Insights:

Users with a high sleep variation may have inconsistent sleep schedules, which could affect their overall health and well-being.

Users with low sleep variation show more regular sleep patterns.

The average sleep duration helps understand overall sleep adequacy per user.

This analysis can help identify individuals who may benefit from sleep pattern interventions.



<h3>2. Total calories burned by each user</h3>

In [12]:
query = """
SELECT Id, SUM(Calories) AS total_calories
FROM dailyActivity
GROUP BY Id
ORDER BY total_calories DESC
"""
pd.read_sql_query(query, conn)


,Id,total_calories
0,8378563200,106534
1,8877689391,106028
2,5577150313,100789
3,4388161847,95910
4,4702921684,91932
5,8053475328,91320
6,8583815059,84693
7,1644430081,84339
8,7086361926,79557
9,2022484408,77809


Explanation:

This query calculates the total calories burned (total_calories) by each user (Id) based on their daily activity records from the dailyActivity table. It sums the Calories column for each user and orders the results in descending order, so users who burned the most calories appear first.

Insights:

Users with higher total calorie expenditure likely have more active lifestyles.

This data can help identify the most physically active users and compare energy expenditure across the user base.

Further analysis can correlate calorie burn with other factors such as steps taken, distance traveled, or sleep quality to get a holistic view of health and fitness.

<h3>3. Average daily steps per user</h3>

In [13]:
query = """
SELECT Id, AVG(TotalSteps) AS avg_steps
FROM dailyActivity
GROUP BY Id
ORDER BY avg_steps DESC
"""
pd.read_sql_query(query, conn)


,Id,avg_steps
0,8877689391,16040.032258
1,8053475328,14763.290323
2,1503960366,12116.741935
3,2022484408,11370.645161
4,7007744171,11323.423077
5,3977333714,10984.566667
6,4388161847,10813.935484
7,6962181067,9794.806452
8,2347167796,9519.666667
9,7086361926,9371.774194


Explanation:

This query calculates the average daily steps (avg_steps) taken by each user (Id) from the dailyActivity table. It groups the data by user and computes the mean of TotalSteps per user, then sorts the users in descending order by their average daily steps.

Insights:

Users with higher average steps are likely more physically active.

Comparing average steps across users helps identify activity patterns and fitness levels.

This metric can be useful for setting personalized activity goals or assessing adherence to recommended physical activity guidelines.

Combined with other data like calories burned or sleep, this can provide a comprehensive picture of overall health.



<h3>4. Top 10 most active users by VeryActiveMinutes</h3>

In [14]:
query = """
SELECT Id, AVG(VeryActiveMinutes) AS avg_active_minutes
FROM dailyActivity
GROUP BY Id
ORDER BY avg_active_minutes DESC
LIMIT 10
"""
pd.read_sql_query(query, conn)


,Id,avg_active_minutes
0,5577150313,87.333333
1,8053475328,85.161290
2,8877689391,66.064516
3,8378563200,58.677419
4,7086361926,42.580645
5,1503960366,38.709677
6,2022484408,36.290323
7,7007744171,31.038462
8,5553957443,23.419355
9,4388161847,23.161290


Explanation:

This query calculates the average number of very active minutes (avg_active_minutes) per day for each user (Id) from the dailyActivity table. It groups the data by user, computes the average of VeryActiveMinutes, and retrieves the top 10 users with the highest average very active minutes.

Insights:

The top 10 users listed demonstrate the highest levels of intense physical activity on average.

Tracking very active minutes provides insight into users’ engagement in vigorous exercise or activities.

This can help identify highly active individuals and tailor fitness recommendations or interventions accordingly.

Focusing on very active minutes complements other activity metrics such as steps and calories burned to form a full activity profile.



<h3>5. Users with highest sleep-to-bedtime ratio</h3>

In [16]:
query = """
SELECT Id, AVG(CAST(TotalMinutesAsleep AS FLOAT) / NULLIF(TotalTimeInBed, 0)) AS sleep_efficiency
FROM sleepDay
GROUP BY Id
ORDER BY sleep_efficiency DESC
"""
pd.read_sql_query(query, conn)


,Id,sleep_efficiency
0,8053475328,0.984888
1,7086361926,0.971315
2,6962181067,0.961487
3,8792009665,0.960392
4,7007744171,0.957117
5,4702921684,0.952655
6,4319703577,0.947497
7,4388161847,0.947414
8,1927972279,0.947008
9,6775888955,0.942028


Explanation:

This query calculates the average sleep efficiency (sleep_efficiency) for each user (Id) from the sleepDay table. Sleep efficiency is computed as the ratio of TotalMinutesAsleep to TotalTimeInBed for each record, cast to a float for precise division. The NULLIF function prevents division by zero by returning NULL when TotalTimeInBed is zero. The query groups the results by user and orders them in descending order of average sleep efficiency.

Insights:

Sleep efficiency reflects the quality of sleep, indicating the percentage of time spent asleep while in bed.

Higher sleep efficiency values suggest better sleep quality, whereas lower values may indicate difficulties falling or staying asleep.

This metric is important for understanding sleep health beyond just total sleep duration.

Identifying users with low sleep efficiency can help target interventions for improving sleep habits and overall well-being.



<h3>6. Daily step totals per user</h3>

In [17]:
query = """
SELECT Id, ActivityDay, StepTotal
FROM dailySteps
ORDER BY StepTotal DESC
LIMIT 10
"""
pd.read_sql_query(query, conn)


,Id,ActivityDay,StepTotal
0,1624580081,5/1/2016,36019
1,8877689391,4/16/2016,29326
2,8877689391,4/30/2016,27745
3,8877689391,4/27/2016,23629
4,8877689391,4/12/2016,23186
5,8053475328,4/24/2016,22988
6,4388161847,5/7/2016,22770
7,8053475328,4/23/2016,22359
8,2347167796,4/16/2016,22244
9,8053475328,5/8/2016,22026


Explanation:

This query retrieves the top 10 records from the dailySteps table showing users (Id) and their corresponding days (ActivityDay) with the highest step counts (StepTotal). It orders the data by StepTotal in descending order, highlighting the days with maximum steps recorded.

Insights:

The results showcase the highest daily step counts achieved by users, indicating peak activity days.

This information can help identify exceptional activity patterns or special events motivating higher physical activity.

Comparing peak days with regular activity levels can reveal factors influencing user motivation or performance.

Highlighting these days may assist in recognizing trends or outliers in step-based activity data.



<h3>7. Daily calories burned and steps taken together</h3>

In [18]:
query = """
SELECT d.Id, d.ActivityDay, d.Calories, s.StepTotal
FROM dailyCalories d
JOIN dailySteps s
ON d.Id = s.Id AND d.ActivityDay = s.ActivityDay
"""
pd.read_sql_query(query, conn)


,Id,ActivityDay,Calories,StepTotal
0,1503960366,4/12/2016,1985,13162
1,1503960366,4/13/2016,1797,10735
2,1503960366,4/14/2016,1776,10460
3,1503960366,4/15/2016,1745,9762
4,1503960366,4/16/2016,1863,12669
...,...,...,...,...
935,8877689391,5/8/2016,2847,10686
936,8877689391,5/9/2016,3710,20226
937,8877689391,5/10/2016,2832,10733
938,8877689391,5/11/2016,3832,21420


Explanation:

This query performs an inner join between the dailyCalories (d) and dailySteps (s) tables, matching records based on the same user (Id) and the same day (ActivityDay). It retrieves each user's calories burned (Calories) along with their total steps (StepTotal) for corresponding days.

Insights:

By combining calorie expenditure and step count data for the same days, this query provides a detailed view of daily physical activity and energy output.

Analyzing the relationship between steps taken and calories burned can help validate fitness tracking accuracy and understand activity intensity.

This joined data can be used to identify trends, such as whether higher step counts consistently lead to higher calorie burn.

Such insights are useful for personalized fitness recommendations and for correlating activity patterns with health outcomes.



<h3>8. Compare very active distance vs calories</h3>

In [19]:
query = """
SELECT Id, AVG(VeryActiveDistance) AS active_dist, AVG(Calories) AS avg_calories
FROM dailyActivity
GROUP BY Id
ORDER BY active_dist DESC
"""
pd.read_sql_query(query, conn)


,Id,active_dist,avg_calories
0,8053475328,8.514839,2945.806452
1,8877689391,6.637419,3420.258065
2,5577150313,3.113667,3359.633333
3,1503960366,2.858387,1816.419355
4,7086361926,2.781290,2566.354839
5,8378563200,2.503548,3436.580645
6,2022484408,2.421613,2509.967742
7,7007744171,2.415000,2544.000000
8,8253242879,2.214210,1788.000000
9,4388161847,1.719355,3093.870968


Explanation:

This query calculates the average very active distance (active_dist) and the average calories burned (avg_calories) per user (Id) from the dailyActivity table. It groups the data by user, computes averages for both metrics, and orders the users by their average very active distance in descending order.

Insights:

Users with higher average very active distances tend to engage in more sustained or intense physical activities.

Comparing average very active distance with average calories burned helps understand the relationship between activity intensity and energy expenditure.

This information can help identify highly active users and assess the effectiveness of their physical activities.

The combined metrics provide a more comprehensive picture of users’ activity levels than either metric alone.



<h3>9. Hourly calorie peaks (by hour of day)</h3>

In [20]:
query = """
SELECT strftime('%H', ActivityHour) AS hour, AVG(Calories) AS avg_calories
FROM hourlyCalories
GROUP BY hour
ORDER BY avg_calories DESC
"""
pd.read_sql_query(query, conn)


,hour,avg_calories
0,None,97.38676


Explanation:

This query analyzes calorie burn patterns by hour of the day. It extracts the hour component (%H) from the ActivityHour timestamp in the hourlyCalories table, then calculates the average calories burned (avg_calories) during each hour. The results are grouped by hour and sorted by average calories burned in descending order.

Insights:

The query reveals the hours during which users typically burn the most calories.

Peaks in average calorie burn can indicate common active periods, such as morning or evening exercise times.

Understanding hourly calorie burn patterns helps tailor activity recommendations and optimize workout scheduling.

This temporal analysis can assist in identifying lifestyle habits and periods of inactivity.



<h3>10. Total METs per user</h3>

In [21]:
query = """
SELECT Id, SUM(METs) AS total_mets
FROM minuteMETsNarrow
GROUP BY Id
ORDER BY total_mets DESC
"""
pd.read_sql_query(query, conn)


,Id,total_mets
0,8877689391,866178
1,5577150313,796171
2,8053475328,750596
3,2022484408,747732
4,4388161847,722167
5,1503960366,717081
6,8378563200,707056
7,7086361926,700993
8,6962181067,684724
9,4558609924,677866


Explanation:

This query calculates the total Metabolic Equivalent of Task (METs) value (total_mets) for each user (Id) by summing the METs recorded in the minuteMETsNarrow table. The results are grouped by user and ordered in descending order of total METs, highlighting users with the highest overall activity intensity.

Insights:

METs quantify the energy cost of physical activities, so a higher total METs value indicates more intense or prolonged activity.

This metric provides a standardized way to compare activity levels across users beyond just steps or calories.

Identifying users with high total METs can help target fitness programs and monitor health improvements.

Total METs can be combined with other metrics like calories and active minutes for comprehensive activity profiling.



 <h3>11. Users with high heart rate values</h3>

In [23]:
query = """
SELECT Id, MAX(Value) AS max_heart_rate
FROM heartRate
GROUP BY Id
ORDER BY max_heart_rate DESC
LIMIT 5
"""
pd.read_sql_query(query, conn)


,Id,max_heart_rate
0,2022484408,203
1,4558609924,199
2,2347167796,195
3,4020332650,191
4,6117666160,189


Explanation:

This query retrieves the maximum recorded heart rate (max_heart_rate) for each user (Id) from the heartRate table. It groups the data by user, selects the highest heart rate value per user, and returns the top 5 users with the highest maximum heart rates.

Insights:

The maximum heart rate reflects peak cardiovascular exertion during physical activity or stress.

Users with very high max heart rates might be engaging in intense workouts or could have underlying health considerations.

This data can help identify individuals with exceptional fitness levels or those who may require monitoring for heart health.

Comparing max heart rate values alongside activity data can provide insights into exercise intensity and cardiovascular response.



<h3>12. Total calories burned per hour</h3>

In [24]:
query = """
SELECT ActivityHour, SUM(Calories) AS total_calories
FROM hourlyCalories
GROUP BY ActivityHour
ORDER BY total_calories DESC
LIMIT 3
"""
pd.read_sql_query(query, conn)


,ActivityHour,total_calories
0,4/27/2016 6:00:00 PM,4987
1,4/23/2016 1:00:00 PM,4926
2,4/19/2016 4:00:00 PM,4712


Explanation:

This query calculates the total calories burned (total_calories) for each recorded hour (ActivityHour) in the hourlyCalories table. It groups the data by the hour timestamp and orders the hours by total calories burned in descending order, retrieving the top 3 hours with the highest calorie expenditure.

Insights:

The query identifies the specific hours during which the greatest amount of calories were burned across all users.

These peak calorie burn hours may correspond to periods of high physical activity or group events.

Understanding peak activity hours can help optimize workout scheduling and promote activity during less active periods.

This temporal analysis complements individual-level insights to understand community or population activity trends.



<h3>13. Correlation data between calories and step total</h3>

In [ ]:
query = """
SELECT a.Calories, s.StepTotal
FROM dailyCalories a
JOIN dailySteps s
ON a.Id = s.Id AND a.ActivityDay = s.ActivityDay
"""
pd.read_sql_query(query, conn)


Explanation:

This query performs an inner join between the dailyCalories (a) and dailySteps (s) tables, matching records by user (Id) and day (ActivityDay). It retrieves paired values of calories burned (Calories) and total steps taken (StepTotal) for the same days, allowing direct comparison of these two related metrics.

Insights:

This combined data enables analysis of the relationship between physical activity (steps) and energy expenditure (calories).

Typically, higher step counts are associated with increased calorie burn, but the exact correlation can vary based on activity intensity and individual metabolism.

This dataset can be used to perform correlation analysis, helping validate the consistency and accuracy of the fitness tracking data.

Understanding this relationship can inform personalized health and fitness recommendations.



<h3>14. Average intensity by hour</h3>

In [4]:
query = """
SELECT strftime('%H', ActivityHour) AS hour, AVG(AverageIntensity) AS avg_intensity
FROM hourlyIntensities
GROUP BY hour
ORDER BY avg_intensity DESC
"""
pd.read_sql_query(query, conn)


,hour,avg_intensity
0,None,0.200589


Explanation:

This query extracts the hour component (%H) from the ActivityHour timestamp in the hourlyIntensities table and calculates the average activity intensity (avg_intensity) for each hour. The data is grouped by hour and ordered in descending order of average intensity, highlighting the hours with the highest average activity intensity.

Insights:

The results reveal at which hours users tend to engage in the most intense physical activities.

Identifying peak intensity hours can help tailor workout plans or suggest optimal times for exercise.

This hourly intensity pattern complements other activity metrics such as calories burned and steps to give a fuller picture of user activity levels throughout the day.

Such temporal analysis can assist in promoting healthier lifestyle habits by encouraging activity during low-intensity periods.



<h3>15. User BMI and calories burned</h3>

In [5]:
query = """
SELECT w.Id, w.BMI, d.Calories
FROM weightLog w
JOIN dailyActivity d
ON w.Id = d.Id AND w.Date = d.ActivityDate
"""
pd.read_sql_query(query, conn)


,Id,BMI,Calories


Explanation:

This query performs an inner join between the weightLog (w) and dailyActivity (d) tables, matching records based on user (Id) and corresponding dates (Date in weightLog and ActivityDate in dailyActivity). It retrieves each user’s Body Mass Index (BMI) alongside their daily calories burned (Calories).

Insights:

By combining BMI data with daily calorie expenditure, this query enables analysis of how users’ body composition relates to their activity levels.

It helps identify whether users with higher or lower BMI tend to burn more calories.

This data can be used to explore correlations between BMI and energy expenditure, useful for tailoring weight management and fitness programs.

Understanding this relationship aids in designing personalized health interventions that consider both body metrics and activity.



<h3>16. Sleep hours vs calories (joined)</h3>

In [6]:
query = """
SELECT s.Id, s.SleepDay, s.TotalMinutesAsleep, d.Calories
FROM sleepDay s
JOIN dailyActivity d
ON s.Id = d.Id AND s.SleepDay = d.ActivityDate
"""
pd.read_sql_query(query, conn)


,Id,SleepDay,TotalMinutesAsleep,Calories


Explanation:

This query performs an inner join between the sleepDay (s) and dailyActivity (d) tables, matching records by user (Id) and date (SleepDay in sleepDay and ActivityDate in dailyActivity). It retrieves each user’s total minutes asleep (TotalMinutesAsleep) alongside their calories burned (Calories) on the same day.

Insights:

This combined data allows analysis of the relationship between sleep duration and daily physical activity (calorie expenditure).

It can help explore whether longer or better-quality sleep correlates with higher or lower activity levels.

Understanding this relationship is useful for promoting balanced health habits, as both sleep and activity impact overall wellness.

The data can inform personalized recommendations for optimizing sleep and activity patterns.



<h3> 17. Light vs very active minutes per user</h3>

In [7]:
query = """
SELECT Id,
       AVG(LightlyActiveMinutes) AS avg_light,
       AVG(VeryActiveMinutes) AS avg_very
FROM dailyActivity
GROUP BY Id
"""
pd.read_sql_query(query, conn)


,Id,avg_light,avg_very
0,1503960366,219.935484,38.709677
1,1624580081,153.483871,8.677419
2,1644430081,178.466667,9.566667
3,1844505072,115.451613,0.129032
4,1927972279,38.580645,1.322581
5,2022484408,257.451613,36.290323
6,2026352035,256.645161,0.096774
7,2320127002,198.193548,1.354839
8,2347167796,252.500000,13.500000
9,2873212765,308.000000,14.096774


Explanation:

This query calculates the average lightly active minutes (avg_light) and very active minutes (avg_very) per user (Id) from the dailyActivity table. It groups the data by user and computes the mean of both activity intensity levels, providing insight into users’ overall activity distribution.

Insights:

LightlyActiveMinutes reflects low-intensity activities such as casual walking or light chores, while VeryActiveMinutes indicates vigorous exercises like running or intense workouts.

Comparing these averages helps understand each user's typical activity intensity profile.

Users with higher very active minutes likely engage in more strenuous exercise, whereas those with more lightly active minutes might maintain moderate activity levels.

This data is valuable for tailoring personalized fitness plans and tracking changes in activity intensity over time.



<h3>18. Max daily steps</h3>

In [9]:
query = """
SELECT Id, MAX(TotalSteps) AS max_steps
FROM dailyActivity
GROUP BY Id
"""
pd.read_sql_query(query, conn)


,Id,max_steps
0,1503960366,18134
1,1624580081,36019
2,1644430081,18213
3,1844505072,8054
4,1927972279,3790
5,2022484408,18387
6,2026352035,12357
7,2320127002,10725
8,2347167796,22244
9,2873212765,9685


Explanation:

This query retrieves the maximum number of steps (max_steps) taken in a single day by each user (Id) from the dailyActivity table. It groups the data by user and selects the highest daily step count for each user.

Insights:

The maximum steps value indicates the peak daily activity level achieved by each user.

This metric can help identify users' most active days, which might correspond to special events, workouts, or lifestyle changes.

Understanding peak activity days can assist in motivating users to reach or surpass their highest activity levels.

Comparing max steps alongside average or total steps provides a fuller picture of users’ activity patterns.

<h3>19. Most intense minute based on METs</h3>

In [10]:
query = """
SELECT Id, ActivityMinute, METs
FROM minuteMETsNarrow
ORDER BY METs DESC
LIMIT 5
"""
pd.read_sql_query(query, conn)


,Id,ActivityMinute,METs
0,2873212765,5/7/2016 7:43:00 AM,157
1,2873212765,5/7/2016 7:44:00 AM,153
2,2873212765,4/23/2016 8:07:00 AM,149
3,2022484408,4/21/2016 4:33:00 PM,146
4,2022484408,4/21/2016 4:34:00 PM,144


Explanation:

This query retrieves the top 5 records from the minuteMETsNarrow table with the highest Metabolic Equivalent of Task (METs) values (METs). It selects the user (Id), the specific minute of activity (ActivityMinute), and the METs value, ordering the results in descending order to highlight the most intense activity minutes.

Insights:

METs quantify the energy cost of physical activities; higher values represent more intense effort.

The top 5 highest METs records indicate moments of peak physical exertion for users.

Analyzing these intense activity periods can help understand users’ high-effort workouts or strenuous daily tasks.

This data is useful for identifying peak performance and designing targeted fitness programs.



<h3>20. Minute-by-minute steps > 100</h3>

In [11]:
query = """
SELECT Id, ActivityMinute, Steps
FROM minuteStepsNarrow
WHERE Steps > 100
ORDER BY Steps DESC
"""
pd.read_sql_query(query, conn)


,Id,ActivityMinute,Steps
0,8053475328,4/30/2016 3:44:00 PM,220
1,4558609924,4/27/2016 6:46:00 PM,207
2,3977333714,4/29/2016 10:47:00 PM,190
3,3977333714,5/4/2016 11:59:00 PM,189
4,3977333714,4/29/2016 10:43:00 PM,188
...,...,...,...
14587,8877689391,5/11/2016 12:46:00 PM,101
14588,8877689391,5/11/2016 12:50:00 PM,101
14589,8877689391,5/12/2016 8:14:00 AM,101
14590,8877689391,5/12/2016 12:24:00 PM,101


Explanation:

This query retrieves records from the minuteStepsNarrow table where the number of steps taken in a single minute (Steps) exceeds 100. It selects the user (Id), the specific minute of activity (ActivityMinute), and the steps count, ordering the results in descending order by steps to highlight the most active minutes.

Insights:

Minutes with over 100 steps represent short bursts of high activity or intense movement.

Identifying these high-step minutes helps capture peak activity moments that may correspond to exercise or brisk walking.

This granular data provides insight into how users accumulate their daily steps, distinguishing between steady activity and short intense bursts.

Such analysis can inform personalized activity goals and reveal patterns in users’ daily movement habits.



<h3>21. Hourly steps sum per user

</h3>

In [12]:
query = """
SELECT Id, strftime('%Y-%m-%d %H', ActivityHour) AS hour_block, SUM(StepTotal) AS total_steps
FROM hourlySteps
GROUP BY Id, hour_block
ORDER BY total_steps DESC
LIMIT 10
"""
pd.read_sql_query(query, conn)


,Id,hour_block,total_steps
0,8877689391,None,495623
1,8053475328,None,457463
2,1503960366,None,374546
3,2022484408,None,351712
4,3977333714,None,328244
5,4388161847,None,319765
6,6962181067,None,303621
7,7007744171,None,294409
8,7086361926,None,287503
9,8378563200,None,269705


Explanation:

This query aggregates step data from the hourlySteps table by user (Id) and hourly time blocks (hour_block), where each block represents a specific year, month, day, and hour. It sums the total steps (total_steps) taken by each user during each hour, then orders the results in descending order to identify the top 10 highest step counts recorded within any hourly block.

Insights:

The query highlights the most active one-hour periods for users, showing when they achieved peak step counts.

These peak hour blocks might correspond to workouts, commutes, or other high-activity routines.

Understanding these hourly step spikes can help optimize daily activity scheduling and encourage consistent movement.

This detailed temporal analysis provides valuable context beyond daily totals, allowing for finer granularity in activity tracking.

<h3> 22. Top 5 longest sleep records
</h3>


In [13]:
query = """
SELECT Id, SleepDay, TotalTimeInBed
FROM sleepDay
ORDER BY TotalTimeInBed DESC
LIMIT 5
"""
pd.read_sql_query(query, conn)


,Id,SleepDay,TotalTimeInBed
0,1644430081,5/2/2016 12:00:00 AM,961
1,1844505072,4/15/2016 12:00:00 AM,961
2,1844505072,4/30/2016 12:00:00 AM,961
3,1844505072,5/1/2016 12:00:00 AM,961
4,5553957443,4/30/2016 12:00:00 AM,843


Explanation:

This query retrieves the top 5 records from the sleepDay table with the longest time spent in bed (TotalTimeInBed). It selects the user (Id), the date of the sleep record (SleepDay), and the total time in bed, ordering the results in descending order by total time in bed.

Insights:

The query identifies users and specific days when they spent the most time in bed.

Longer time in bed may indicate extended rest, but does not necessarily equate to better sleep quality.

This data can be used alongside sleep efficiency or total minutes asleep to assess sleep quality versus duration.

Understanding extremes in time spent in bed helps identify potential sleep pattern anomalies or lifestyle factors affecting rest.



<h3>23. Total calories per day (all users)</h3>

In [14]:
query = """
SELECT ActivityDay, SUM(Calories) AS total_calories
FROM dailyCalories
GROUP BY ActivityDay
ORDER BY ActivityDay
"""
pd.read_sql_query(query, conn)


,ActivityDay,total_calories
0,4/12/2016,78893
1,4/13/2016,75459
2,4/14/2016,77761
3,4/15/2016,77721
4,4/16/2016,76574
5,4/17/2016,71391
6,4/18/2016,74668
7,4/19/2016,75491
8,4/20/2016,76647
9,4/21/2016,77500


Explanation:

This query calculates the total calories burned (total_calories) for each day (ActivityDay) by summing calories recorded in the dailyCalories table. The results are grouped by day and ordered chronologically.

Insights:

This query reveals daily trends in overall calorie expenditure across all users.

Observing calorie totals over time can help identify patterns such as weekdays vs weekends or seasonal activity changes.

The data can highlight periods of increased or decreased collective activity, useful for planning group fitness challenges or interventions.

Analyzing total daily calories alongside other metrics like steps or sleep can provide comprehensive insights into user behavior.



<h3>24. Average BMI per user
</h3>

In [15]:
query = """
SELECT Id, AVG(BMI) AS avg_bmi
FROM weightLog
GROUP BY Id
"""
pd.read_sql_query(query, conn)


,Id,avg_bmi
0,1503960366,22.650000
1,1927972279,47.540001
2,2873212765,21.570001
3,4319703577,27.415000
4,4558609924,27.214000
5,5577150313,28.000000
6,6962181067,24.028000
7,8877689391,25.487083


Explanation:

This query calculates the average Body Mass Index (avg_bmi) for each user (Id) by averaging their BMI values recorded in the weightLog table. The data is grouped by user to provide an overall average BMI per individual.

Insights:

The average BMI gives an overview of each user’s typical body composition over time.

Tracking average BMI helps monitor weight trends and potential health risks related to underweight, overweight, or obesity.

This metric can be used alongside activity and calorie data to analyze how lifestyle factors influence body composition.

Understanding average BMI supports personalized health recommendations and fitness goals.



<h3>25. Fairly Active Minutes vs Lightly Active Minutes</h3>

In [16]:
query = """
SELECT Id,
       AVG(FairlyActiveMinutes) AS avg_fairly_active,
       AVG(LightlyActiveMinutes) AS avg_lightly_active
FROM dailyActivity
GROUP BY Id
ORDER BY avg_fairly_active DESC
"""
pd.read_sql_query(query, conn)


,Id,avg_fairly_active,avg_lightly_active
0,3977333714,61.266667,174.766667
1,5577150313,29.833333,147.933333
2,4702921684,26.032258,237.483871
3,7086361926,25.354839,143.838710
4,8583815059,22.193548,138.290323
5,1644430081,21.366667,178.466667
6,2347167796,20.555556,252.500000
7,4388161847,20.354839,229.354839
8,2022484408,19.354839,257.451613
9,1503960366,19.161290,219.935484


Explanation:

This query calculates the average number of fairly active minutes (avg_fairly_active) and lightly active minutes (avg_lightly_active) for each user (Id) from the dailyActivity table. The results are grouped by user and ordered in descending order of fairly active minutes, highlighting users with the highest average moderate-intensity activity.

Insights:

FairlyActiveMinutes represent moderate-intensity activities, such as brisk walking or light cycling.

LightlyActiveMinutes capture low-intensity activities like casual walking or household tasks.

Ordering by fairly active minutes highlights the most consistently moderately active users.

Comparing these averages provides insight into users’ activity intensity distribution and lifestyle patterns.

This data is valuable for designing tailored fitness programs to encourage balanced activity levels and improve overall health.



<h1>step3:Advanced SQL Analysis</h1>

<h3>26.Days when users burned more than their average calories</h3>

In [17]:
query = """
SELECT a.Id, a.ActivityDate, a.Calories
FROM dailyActivity a
JOIN (
    SELECT Id, AVG(Calories) AS avg_cal
    FROM dailyActivity
    GROUP BY Id
) b ON a.Id = b.Id
WHERE a.Calories > b.avg_cal
"""
pd.read_sql_query(query, conn)


,Id,ActivityDate,Calories
0,1503960366,4/12/2016,1985
1,1503960366,4/16/2016,1863
2,1503960366,4/18/2016,1921
3,1503960366,4/19/2016,2035
4,1503960366,4/22/2016,1827
...,...,...,...
512,8877689391,5/4/2016,3477
513,8877689391,5/6/2016,4015
514,8877689391,5/7/2016,4142
515,8877689391,5/9/2016,3710


Explanation:

This query retrieves records from the dailyActivity table for days when a user’s calorie expenditure (Calories) exceeds their personal average calorie burn. It first calculates the average calories burned per user (avg_cal) using a subquery, then joins this with the main table to filter days where the calorie burn is above that average.

Insights:

This query helps identify days when users were more active than usual, burning more calories than their average.

Analyzing these above-average days can reveal patterns such as workouts, events, or lifestyle changes leading to higher activity.

Tracking high-activity days supports understanding users’ motivation and consistency in physical activity.

This information can be leveraged to encourage users to maintain or increase their active days for better health outcomes.



<h3>27.average of daily steps over 3 days per user</h3>

In [18]:
query = """
SELECT Id, ActivityDate, TotalSteps,
       AVG(TotalSteps) OVER (PARTITION BY Id ORDER BY ActivityDate ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS rolling_avg_steps
FROM dailyActivity
"""
pd.read_sql_query(query, conn)


,Id,ActivityDate,TotalSteps,rolling_avg_steps
0,1503960366,4/12/2016,13162,13162.000000
1,1503960366,4/13/2016,10735,11948.500000
2,1503960366,4/14/2016,10460,11452.333333
3,1503960366,4/15/2016,9762,10319.000000
4,1503960366,4/16/2016,12669,10963.666667
...,...,...,...,...
935,8877689391,5/5/2016,14055,14355.333333
936,8877689391,5/6/2016,21727,17991.666667
937,8877689391,5/7/2016,12332,16038.000000
938,8877689391,5/8/2016,10686,14915.000000


Explanation:

This query calculates a 3-day rolling average of total steps (rolling_avg_steps) for each user (Id) using a window function. For every record in dailyActivity, it computes the average of the current day’s steps plus the two preceding days' steps, ordered by ActivityDate. This provides a smoothed trend of step activity over time for each user.

Insights:

Rolling averages help smooth out daily fluctuations and highlight longer-term activity trends.

By looking at 3-day averages, short-term variations like rest days or unusually active days become less pronounced.

This approach can identify gradual increases or decreases in user activity more clearly than daily values alone.

Such trends are useful for monitoring progress, setting fitness goals, and detecting early signs of changes in behavior or health.



<h3>28.Users with above average sleep but below average steps</h3>

In [19]:
query = """
SELECT s.Id
FROM (
    SELECT Id, AVG(TotalMinutesAsleep) AS avg_sleep
    FROM sleepDay
    GROUP BY Id
) s
JOIN (
    SELECT Id, AVG(TotalSteps) AS avg_steps
    FROM dailyActivity
    GROUP BY Id
) d ON s.Id = d.Id
WHERE s.avg_sleep > 400 AND d.avg_steps < 5000
"""
pd.read_sql_query(query, conn)


,Id
0,1844505072
1,1927972279
2,8792009665


Explanation:

This query identifies users (Id) who, on average, sleep more than 400 minutes (about 6 hours 40 minutes) per day but take fewer than 5,000 steps daily. It does this by calculating average total minutes asleep (avg_sleep) from the sleepDay table and average total steps (avg_steps) from the dailyActivity table, then joining on user ID and filtering based on these thresholds.

Insights:

The query highlights users who may have relatively high sleep duration but low physical activity.

This group might represent individuals with sedentary lifestyles or potential health concerns related to inactivity.

Understanding this subset is important for targeting interventions to encourage increased physical activity while maintaining good sleep habits.

Such analysis can help tailor wellness programs that balance rest and movement for overall health improvement.

<h3>29.Rank users based on total distance</h3>

In [20]:
query = """
SELECT Id, SUM(TotalDistance) AS total_distance,
       RANK() OVER (ORDER BY SUM(TotalDistance) DESC) AS rank_distance
FROM dailyActivity
GROUP BY Id
"""
pd.read_sql_query(query, conn)


,Id,total_distance,rank_distance
0,8877689391,409.599997,1
1,8053475328,355.729997,2
2,4388161847,260.190003,3
3,2022484408,250.609998,4
4,1503960366,242.099999,5
5,3977333714,225.509998,6
6,4702921684,215.610000,7
7,8378563200,214.320002,8
8,7007744171,208.399999,9
9,6962181067,204.160001,10


Explanation:

This query calculates the total distance traveled (total_distance) by each user (Id) by summing the TotalDistance values from the dailyActivity table. It also assigns a rank (rank_distance) to each user based on their total distance in descending order, with the highest total distance ranked first. The results are grouped by user.

Insights:

The total distance metric reflects overall user mobility and physical activity level.

Ranking users by total distance helps identify the most active individuals in terms of distance covered.

This can be useful for recognizing top performers or motivating users to increase their distance.

Tracking total distance alongside other metrics like steps and calories can give a comprehensive picture of physical activity.



<h3>30.Daily weight loss tracking</h3>

In [22]:
query = """
SELECT Id, Date, WeightKg,
       WeightKg - LAG(WeightKg, 1) OVER (PARTITION BY Id ORDER BY Date) AS daily_change
FROM weightLog
ORDER BY Id, Date
"""
pd.read_sql_query(query, conn)


,Id,Date,WeightKg,daily_change
0,1503960366,5/2/2016 11:59:59 PM,52.599998,NaN
1,1503960366,5/3/2016 11:59:59 PM,52.599998,0.000000
2,1927972279,4/13/2016 1:08:52 AM,133.500000,NaN
3,2873212765,4/21/2016 11:59:59 PM,56.700001,NaN
4,2873212765,5/12/2016 11:59:59 PM,57.299999,0.599998
...,...,...,...,...
62,8877689391,5/3/2016 6:49:41 AM,84.900002,0.900002
63,8877689391,5/4/2016 6:48:22 AM,84.400002,-0.500000
64,8877689391,5/6/2016 6:43:35 AM,85.000000,0.599998
65,8877689391,5/8/2016 7:35:53 AM,85.400002,0.400002


Explanation:

This query retrieves weight records (WeightKg) for each user (Id) from the weightLog table along with the date of the record (Date). It calculates the daily change in weight (daily_change) by subtracting the previous day's weight from the current day's weight using the LAG window function, partitioned by user and ordered by date. The results are ordered by user and date.

Insights:

The daily change in weight helps track short-term weight fluctuations for each user.

Monitoring daily weight changes can reveal trends like gradual weight loss, gain, or stability.

This data is useful for users aiming to manage weight or assess the impact of diet and exercise routines.

Identifying sudden large changes may flag data anomalies or notable health events.



<h3>31.Activity level classification</h3>

In [24]:
query = """
SELECT Id, ActivityDate,
       CASE 
         WHEN TotalSteps >= 10000 THEN 'High'
         WHEN TotalSteps >= 5000 THEN 'Moderate'
         ELSE 'Low'
       END AS activity_level
FROM dailyActivity
"""
pd.read_sql_query(query, conn)


,Id,ActivityDate,activity_level
0,1503960366,4/12/2016,High
1,1503960366,4/13/2016,High
2,1503960366,4/14/2016,High
3,1503960366,4/15/2016,Moderate
4,1503960366,4/16/2016,High
...,...,...,...
935,8877689391,5/8/2016,High
936,8877689391,5/9/2016,High
937,8877689391,5/10/2016,High
938,8877689391,5/11/2016,High


Explanation:

This query classifies daily step counts (TotalSteps) for each user (Id) and date (ActivityDate) into three activity levels:

High: 10,000 steps or more,

Moderate: between 5,000 and 9,999 steps,

Low: fewer than 5,000 steps.

The classification uses a CASE statement to create an activity_level label for each day.

Insights:

This categorization helps quickly understand users’ daily activity intensity based on step goals.

The thresholds align with common physical activity guidelines, where 10,000 steps are often considered a target for an active day.

Tracking activity levels over time can reveal trends in user behavior, adherence to fitness goals, and periods of inactivity.

Such classifications are useful for personalized feedback, challenges, or intervention planning

<h3>32.Average sleep by weekday</h3>

In [25]:
query = """
SELECT strftime('%w', SleepDay) AS weekday,
       AVG(TotalMinutesAsleep) AS avg_sleep
FROM sleepDay
GROUP BY weekday
ORDER BY weekday
"""
pd.read_sql_query(query, conn)


,weekday,avg_sleep
0,None,419.467312


Explanation:

This query calculates the average total minutes asleep (avg_sleep) for each day of the week (weekday) from the sleepDay table. It uses the strftime('%w', SleepDay) function to extract the weekday as a number (0 = Sunday, 1 = Monday, ..., 6 = Saturday), groups the data by weekday, and orders the results from Sunday to Saturday.

Insights:

The query reveals sleep duration patterns across different days of the week.

This helps identify if users tend to sleep more or less on weekends versus weekdays.

Such analysis can highlight lifestyle or work-related influences on sleep habits.

Understanding weekly sleep trends is useful for recommending adjustments to improve rest quality.



<h3>33.Percentage of time asleep in bed</h3>

In [26]:
query = """
SELECT Id, SleepDay,
       ROUND(CAST(TotalMinutesAsleep AS FLOAT) / TotalTimeInBed * 100, 2) AS sleep_efficiency
FROM sleepDay
WHERE TotalTimeInBed > 0
ORDER BY sleep_efficiency DESC
"""
pd.read_sql_query(query, conn)


,Id,SleepDay,sleep_efficiency
0,4020332650,4/16/2016 12:00:00 AM,100.00
1,7086361926,4/25/2016 12:00:00 AM,99.78
2,6962181067,4/30/2016 12:00:00 AM,99.53
3,6962181067,4/24/2016 12:00:00 AM,99.42
4,7086361926,4/30/2016 12:00:00 AM,99.18
...,...,...,...
408,3977333714,4/25/2016 12:00:00 AM,56.10
409,3977333714,4/16/2016 12:00:00 AM,55.49
410,3977333714,4/19/2016 12:00:00 AM,55.44
411,3977333714,5/7/2016 12:00:00 AM,53.50


Explanation:

This query calculates the sleep efficiency for each record in the sleepDay table, defined as the percentage of time spent asleep (TotalMinutesAsleep) out of the total time spent in bed (TotalTimeInBed). It filters out records where TotalTimeInBed is zero to avoid division errors. The sleep efficiency is rounded to two decimal places and the results are ordered from highest to lowest efficiency.

Insights:

Sleep efficiency is a key metric to assess sleep quality — higher percentages indicate more restful and uninterrupted sleep.

This metric helps differentiate between users who spend a lot of time in bed but sleep poorly versus those with efficient sleep cycles.

Analyzing sleep efficiency alongside total sleep time and activity can offer a fuller picture of health and recovery.

Identifying patterns of low sleep efficiency can prompt investigations into sleep disorders or lifestyle factors affecting rest.



<h3>35.Top 10 most intense minutes</h3>

In [27]:
query = """
SELECT Id, ActivityMinute, Intensity
FROM minuteIntensitiesNarrow
ORDER BY Intensity DESC
LIMIT 10
"""
pd.read_sql_query(query, conn)


,Id,ActivityMinute,Intensity
0,1503960366,4/12/2016 2:51:00 PM,3
1,1503960366,4/12/2016 2:52:00 PM,3
2,1503960366,4/12/2016 2:53:00 PM,3
3,1503960366,4/12/2016 2:57:00 PM,3
4,1503960366,4/12/2016 2:58:00 PM,3
5,1503960366,4/12/2016 3:00:00 PM,3
6,1503960366,4/12/2016 3:01:00 PM,3
7,1503960366,4/12/2016 3:02:00 PM,3
8,1503960366,4/12/2016 3:03:00 PM,3
9,1503960366,4/12/2016 3:04:00 PM,3


Explanation:

This query selects the top 10 records from the minuteIntensitiesNarrow table based on the highest intensity values (Intensity). It retrieves the user ID (Id), the specific minute of activity (ActivityMinute), and the intensity level. The results are ordered in descending order of intensity, showing the most intense activity minutes across users.

Insights:

The query highlights moments of peak physical activity intensity at the minute level.

Understanding these high-intensity bursts can provide insights into users’ workout sessions or strenuous activities.

This data can be useful for analyzing short, intense exercise patterns such as sprints, intervals, or sudden activity spikes.

Identifying peak intensity periods can help tailor fitness programs or monitor exercise effectiveness.







<h3>conclusion</h3>

Through comprehensive analysis of user health and activity data using advanced SQL techniques, we can uncover valuable insights into fitness patterns and behaviors. This integrated approach helps reveal correlations between activity levels, sleep quality, and body metrics, enabling personalized recommendations and better health management. Such data-driven analysis empowers users to make informed decisions for improving their overall wellness.



<h1>Successfully Completed SQL Analysis on Strava Fitness app</h1>